In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import argparse
import itertools
from copy import deepcopy
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image
import torch
import os, sys

In [33]:
SCRIPT_DIR = os.path.dirname(os.path.abspath("/content"))
sys.path.append(os.path.dirname(SCRIPT_DIR))
SCRIPT_DIR = os.path.dirname(os.path.abspath("/content/code"))
sys.path.append(os.path.dirname(SCRIPT_DIR))
sys.path.append(".")

In [34]:
from models_style import Generator
from models_style import Discriminator
from utils import ReplayBuffer
from utils import LambdaLR
from utils import Logger
from utils import weights_init_normal
from datasets import ImageDataset

In [35]:
params = argparse.Namespace()
params.dataset = "facades"
params.num_epochs = 150
params.batch_size = 1
params.lr = 0.0002
params.decay_epoch = 100
params.input_size = 256
params.resize_scale =286
params.crop_size = 256
params.input_nc = 3
params.output_nc = 3
params.dataroot = os.path.join(os.path.abspath('.'), 'datasets/horse2zebra/')
params.device = "cuda" if torch.cuda.is_available() else "cpu"

In [36]:
netG_A2B = Generator(params.input_nc, params.output_nc).to(params.device)
netG_B2A = Generator(params.output_nc, params.input_nc).to(params.device)
netD_A = Discriminator(params.input_nc).to(params.device)
netD_B = Discriminator(params.output_nc).to(params.device)

In [37]:
netG_A2B = netG_A2B.apply(weights_init_normal)
netG_B2A = netG_B2A.apply(weights_init_normal)
netD_A = netD_A.apply(weights_init_normal)
netD_B = netD_B.apply(weights_init_normal)

In [38]:
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

In [39]:
optimizer_G = torch.optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()),
                                lr=params.lr, betas=(0.5, 0.999))
optimizer_D_A = torch.optim.Adam(netD_A.parameters(), lr=params.lr, betas=(0.5, 0.999))
optimizer_D_B = torch.optim.Adam(netD_B.parameters(), lr=params.lr, betas=(0.5, 0.999))

In [40]:
lr_sched_params = LambdaLR(params.num_epochs, 0, params.decay_epoch).step
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=deepcopy(lr_sched_params))
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=deepcopy(lr_sched_params))
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=deepcopy(lr_sched_params))

In [41]:
input_A = torch.Tensor(params.batch_size, params.input_nc, params.crop_size, params.crop_size)
input_B = torch.Tensor(params.batch_size, params.output_nc, params.crop_size, params.crop_size)
target_real =torch.tensor(params.batch_size, requires_grad=False, dtype=torch.float).fill_(1.0).to(params.device)
target_fake = torch.tensor(params.batch_size, requires_grad=False, dtype=torch.float).fill_(0.0).to(params.device)

In [42]:
fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

In [43]:
transforms_ = [ transforms.Resize(int(params.resize_scale), Image.BICUBIC),
                transforms.RandomCrop(params.crop_size),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
dataloader = DataLoader(ImageDataset(params.dataroot, transforms_=transforms_, unaligned=True),
                        batch_size=params.batch_size,
                        shuffle=True,
                        num_workers=4)

In [44]:
logger = Logger(params.num_epochs, len(dataloader))

Setting up a new session...


In [ ]:

###################################

###### Training ######
for epoch in range(0, params.num_epochs):
    for i, batch in enumerate(dataloader):
        # Set model input
        real_A = batch['A'].to(params.device, torch.float)
        real_B = batch['B'].to(params.device, torch.float)

        ###### Generators A2B and B2A ######
        optimizer_G.zero_grad()

        # Identity loss
        # G_A2B(B) should equal B if real B is fed
        same_B = netG_A2B(real_B)
        loss_identity_B = criterion_identity(same_B, real_B)*5.0
        # G_B2A(A) should equal A if real A is fed
        same_A = netG_B2A(real_A)
        loss_identity_A = criterion_identity(same_A, real_A)*5.0

        # GAN loss
        fake_B = netG_A2B(real_A)
        pred_fake = netD_B(fake_B)
        loss_GAN_A2B = criterion_GAN(pred_fake, target_real)

        fake_A = netG_B2A(real_B)
        pred_fake = netD_A(fake_A)
        loss_GAN_B2A = criterion_GAN(pred_fake, target_real)

        # Cycle loss
        recovered_A = netG_B2A(fake_B)
        loss_cycle_ABA = criterion_cycle(recovered_A, real_A)*10.0

        recovered_B = netG_A2B(fake_A)
        loss_cycle_BAB = criterion_cycle(recovered_B, real_B)*10.0

        # Total loss
        loss_G = loss_identity_A + loss_identity_B + loss_GAN_A2B + loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB
        loss_G.backward()

        optimizer_G.step()
        ###################################

        ###### Discriminator A ######
        optimizer_D_A.zero_grad()

        # Real loss
        pred_real = netD_A(real_A)
        loss_D_real = criterion_GAN(pred_real, target_real)

        # Fake loss
        fake_A = fake_A_buffer.push_and_pop(fake_A)
        pred_fake = netD_A(fake_A.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)

        # Total loss
        loss_D_A = (loss_D_real + loss_D_fake)*0.5
        loss_D_A.backward()

        optimizer_D_A.step()
        ###################################

        ###### Discriminator B ######
        optimizer_D_B.zero_grad()

        # Real loss
        pred_real = netD_B(real_B)
        loss_D_real = criterion_GAN(pred_real, target_real)

        # Fake loss
        fake_B = fake_B_buffer.push_and_pop(fake_B)
        pred_fake = netD_B(fake_B.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)

        # Total loss
        loss_D_B = (loss_D_real + loss_D_fake)*0.5
        loss_D_B.backward()

        optimizer_D_B.step()
        ###################################

        # Progress report (http://localhost:8097)
        logger.log({'loss_G': loss_G, 'loss_G_identity': (loss_identity_A + loss_identity_B), 'loss_G_GAN': (loss_GAN_A2B + loss_GAN_B2A),
                    'loss_G_cycle': (loss_cycle_ABA + loss_cycle_BAB), 'loss_D': (loss_D_A + loss_D_B)},
                    images={'real_A': real_A, 'real_B': real_B, 'fake_A': fake_A, 'fake_B': fake_B})
    # Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()

    # Save models checkpoints
    torch.save(netG_A2B.state_dict(), 'output/netG_A2B.pth')
    torch.save(netG_B2A.state_dict(), 'output/netG_B2A.pth')
    torch.save(netD_A.state_dict(), 'output/netD_A.pth')
    torch.save(netD_B.state_dict(), 'output/netD_B.pth')

Epoch 001/150 [1208/1334] -- loss_G: 8.3166 | loss_G_identity: 2.3846 | loss_G_GAN: 0.7926 | loss_G_cycle: 5.1394 | loss_D: 0.4339 -- ETA: 1 day, 12:13:58.847443loss_G: 8.4655 | loss_G_identity: 2.4318 | loss_G_GAN: 0.7909 | loss_G_cycle: 5.2428 | loss_D: 0.4320 -- ETA: 1 day, 12:33:44.697293loss_G: 8.4185 | loss_G_identity: 2.4166 | loss_G_GAN: 0.7912 | loss_G_cycle: 5.2107 | loss_D: 0.4305 -- ETA: 1 day, 12:30:15.950053loss_G: 8.4174 | loss_G_identity: 2.4158 | loss_G_GAN: 0.7927 | loss_G_cycle: 5.2090 | loss_D: 0.4310 -- ETA: 1 day, 12:29:55.464947loss_G: 8.3621 | loss_G_identity: 2.3984 | loss_G_GAN: 0.7938 | loss_G_cycle: 5.1699 | loss_D: 0.4332 -- ETA: 1 day, 12:23:41.364417

Connection to remote host was lost.
Connection to remote host was lost.
on_close() takes 1 positional argument but 3 were given
Connection to remote host was lost.
on_close() takes 1 positional argument but 3 were given
on_close() takes 1 positional argument but 3 were given
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/en

Epoch 001/150 [1209/1334] -- loss_G: 8.3172 | loss_G_identity: 2.3846 | loss_G_GAN: 0.7922 | loss_G_cycle: 5.1405 | loss_D: 0.4340 -- ETA: 1 day, 12:13:50.712975Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Epoch 001/150 [1210/1334] -- loss_G: 8.3175 | loss_G_identity: 2.3848 | loss_G_GAN: 0.7916 | loss_G_cycle: 5.1412 | loss_D: 0.4340 -- ETA: 1 day, 12:13:42.748327Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
-----------------------------------------------------------

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/connectionpool.py", line 398, in _make_

Epoch 001/150 [1211/1334] -- loss_G: 8.3164 | loss_G_identity: 2.3843 | loss_G_GAN: 0.7913 | loss_G_cycle: 5.1408 | loss_D: 0.4342 -- ETA: 1 day, 12:13:32.467563Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------


...

Epoch 001/150 [1276/1334] -- loss_G: 8.2747 | loss_G_identity: 2.3682 | loss_G_GAN: 0.7950 | loss_G_cycle: 5.1115 | loss_D: 0.4329 -- ETA: 1 day, 12:03:12.047116Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/connectionpool.py", line 398, in _make_

Epoch 001/150 [1277/1334] -- loss_G: 8.2746 | loss_G_identity: 2.3682 | loss_G_GAN: 0.7952 | loss_G_cycle: 5.1112 | loss_D: 0.4330 -- ETA: 1 day, 12:03:04.054663Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/home/ubuntu/anaconda3/envs/pytorch_p39_research/lib/python3.9/site-packages/urllib3/connectionpool.py", line 398, in _make_

Epoch 001/150 [1334/1334] -- loss_G: 8.2516 | loss_G_identity: 2.3596 | loss_G_GAN: 0.7972 | loss_G_cycle: 5.0947 | loss_D: 0.4320 -- ETA: 1 day, 12:01:02.210868
Epoch 002/150 [0018/1334] -- loss_G: 7.4893 | loss_G_identity: 2.0792 | loss_G_GAN: 0.8414 | loss_G_cycle: 4.5687 | loss_D: 0.3923 -- ETA: 1 day, 12:18:20.918043